In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

print("Train: {}, Test :{}".format(df_train.shape,df_test.shape))
df_train.head()

Train: (7352, 563), Test :(2947, 563)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [10]:
df_train.Activity.value_counts()

LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: Activity, dtype: int64

In [61]:
len(df_train.subject.unique()),len(df_test.subject.unique())

(21, 9)

In [49]:
"Lets try with both subject and not subject"
#without subject
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
labelEncoder.fit(df_train.Activity)
Y_train = labelEncoder.transform(df_train.Activity)
Y_test = labelEncoder.transform(df_test.Activity)
Y_train[:20]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
      dtype=int64)

In [50]:
encoding = { l:c for l,c in zip(labelEncoder.classes_,labelEncoder.transform(labelEncoder.classes_))}
print(encoding)

{'LAYING': 0, 'SITTING': 1, 'STANDING': 2, 'WALKING': 3, 'WALKING_DOWNSTAIRS': 4, 'WALKING_UPSTAIRS': 5}


In [63]:
X_train = df_train.drop(['subject','Activity'],axis=1)
X_test = df_test.drop(['subject','Activity'],axis=1)
                      
X_train.shape,X_test.shape

((7352, 561), (2947, 561))

In [64]:
#lets train a simple Logistic regression model
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(penalty="l2",max_iter=150,random_state=5)
lg.fit(X_train,Y_train)
_y = lg.predict(X_test)
from sklearn.metrics import accuracy_score, confusion_matrix
print("Accuracy : {}".format(accuracy_score(Y_test,_y)))
print("Confusion matrix \n {}".format(confusion_matrix(Y_test,_y)))

Accuracy : 0.9619952494061758
Confusion matrix 
 [[537   0   0   0   0   0]
 [  0 432  55   0   0   4]
 [  0  13 517   2   0   0]
 [  0   0   0 494   2   0]
 [  0   0   0   4 407   9]
 [  0   0   0  23   0 448]]


In [65]:
#lets train some something more powerful classifier :SVM
from sklearn.svm import SVC
svc = SVC(C=1,random_state=5,tol=0.0001)
svc.fit(X_train,Y_train)
_y = svc.predict(X_test)
print("Accuracy : {}".format(accuracy_score(Y_test,_y)))
print("Confusion Matrix \n {}".format(confusion_matrix(Y_test,_y)))

Accuracy : 0.9256871394638616
Confusion Matrix 
 [[537   0   0   0   0   0]
 [  0 406  83   0   0   2]
 [  0  50 482   0   0   0]
 [  0   0   0 491   5   0]
 [  0   0   0  21 365  34]
 [  0   0   0  21   3 447]]


In [66]:
#lets try emsemble technique
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=50,random_state=5,verbose=1)
rfc.fit(X_train,Y_train)
_y = rfc.predict(X_test)
print("Accuracy : {}".format(accuracy_score(Y_test,_y)))
print("Confusion Matrix \n {}".format(confusion_matrix(Y_test,_y)))

Accuracy : 0.9246691550729556
Confusion Matrix 
 [[537   0   0   0   0   0]
 [  0 438  53   0   0   0]
 [  0  44 488   0   0   0]
 [  0   0   0 480  12   4]
 [  0   0   0  20 357  43]
 [  0   0   0  40   6 425]]


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:   16.3s finished
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.0s finished


In [47]:
#try XGBoost